In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

import pandas as pd

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
# from scipy.io import loadmat

# library written for this exercise providing additional functions for assignment submission, and others
# import utils


# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [4]:

# Cargar el dataset
df = pd.read_csv('/content/drive/MyDrive/Score.csv', sep=",")

diccionario = {"Poor": 0, "Standard": 1, "Good": 2}
df["Credit_Score"] = df["Credit_Score"].replace(diccionario)

diccionario1 = {"No": 0, "Yes": 1, "NM": 2}
df["Payment_of_Min_Amount"] = df["Payment_of_Min_Amount"].replace(diccionario1)

diccionario2 = {"Bad": 0, "Standard": 1, "Good": 2}
df["Credit_Mix"] = df["Credit_Mix"].replace(diccionario2)

df = df.drop("Payment_Behaviour", axis=1)

df.drop_duplicates(inplace=True)
df.dropna(axis=0, thresh=2, inplace=True)  # Elimina filas con n o más valores nulos.

# Seleccionar las características (features) y la variable objetivo (target)
X_train = df.iloc[:80000, df.columns != "Credit_Score"]
y_train = df.iloc[:80000, df.columns.get_loc("Credit_Score")]

X_test = df.iloc[80000:, df.columns != "Credit_Score"]
y_test = df.iloc[80000:, df.columns.get_loc("Credit_Score")]

X = X_train
y = y_train

y = np.array([int(e) for e in y])
y = np.squeeze(y)

print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

print(X)

(80000, 19)
(80000,)
(19960, 19)
(19960,)
       Delay_from_due_date  Num_of_Delayed_Payment  Num_Credit_Inquiries  \
0                      3.0                     7.0                   4.0   
1                      3.0                     7.0                   4.0   
2                      3.0                     7.0                   4.0   
3                      5.0                     4.0                   4.0   
4                      6.0                     4.0                   4.0   
...                    ...                     ...                   ...   
79995                 22.0                    23.0                   6.0   
79996                 22.0                    23.0                   6.0   
79997                 22.0                    23.0                   6.0   
79998                 27.0                    20.0                   6.0   
79999                 23.0                    20.0                   7.0   

       Credit_Utilization_Ratio  Credit_Histo

In [ ]:
#scala cada característica individualmente restando el valor mínimo y luego dividiendo por el rango (es decir, la diferencia entre el máximo y el mínimo)
from sklearn.preprocessing import MinMaxScaler

# Convertir datos a tipo float
X = X.astype(float)

# Crear el objeto MinMaxScaler
scaler = MinMaxScaler()

# Ajustar el scaler a los datos y transformarlos
X_normalized = scaler.fit_transform(X)

print(X_normalized)

In [ ]:
#normalizamos las características
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [ ]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

print(X)
print('Media calculada:', mu)
print('Desviación estandar calculada:', sigma)
print(X_norm)

In [5]:
# Configurando parametros necesario
input_layer_size  = 19  # Entrada de 19 caracteristicas
hidden_layer_size = 10   # 10 unidades ocultas
num_labels = 3          # 3 etiquetas, de 0 a 2

# carga los pesos en las variables Theta1 y Theta2
# weights = loadmat(os.path.join('/content/gdrive/MyDrive/Colab Notebooks/machine learning/data', 'ex4weights.mat'))
# weights = np.array()
pesos = {}
pesos['Theta1'] = np.random.rand(10, 20)
pesos['Theta2'] = np.random.rand(3, 11)
# print(pesos['Theta1'][:].shape)
# print(pesos['Theta2'][:].shape)

# print(weights['Theta1'][:].shape)
# print(weights['Theta2'][:].shape)

# print(weights['Theta1'][0])
# print(np.roll(weights['Theta1'][0], 1, axis=0))
# Theta1 tiene un tamaño de 25x401
# Theta2 tiene un tamañó de 10x26
# Theta1, Theta2 = weights['Theta1'], weights['Theta2']
Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']
# se intercambia la ultima columa con la primera de Theta2, por cuestiones de indices que utiliza MATLAB
# print(Theta2)
# print(np.roll(Theta2, 1, axis=0))

# Theta2 = np.roll(Theta2, 1, axis=0)

# Desenrollar parámetros
print(Theta1.ravel().shape)
print(Theta2.ravel().shape)

nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(200,)
(33,)
(233,)


In [6]:

def sigmoid(z):
    """
    Computes the sigmoid of z.
    """
    return 1.0 / (1.0 + np.exp(-z))

def sigmoidGradient(z):
    """
    Computes the gradient of the sigmoid function for z.
    """
    g = np.zeros(z.shape)
    g = sigmoid(z) * (1 - sigmoid(z))
    return g

def normalize_features(X):
    """
    Normalizes the features of input data X.
    """
    X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    return X_norm

In [8]:
X_norm = normalize_features(X)
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):

    # Reshape nn_params back into the parameters Theta1 and Theta2, the weight matrices
    # for our 2 layer neural network
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size

    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)

    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)

    a3 = sigmoid(a2.dot(Theta2.T))

    # print("-"*20)
    # print(y.shape)
    # print(y.reshape(-1))
    # print("-"*20)
    y_matrix = y.reshape(-1)
    # print(y.shape)
    y_matrix = np.eye(num_labels)[y_matrix]
    # print(y_matrix)

    temp1 = Theta1
    temp2 = Theta2

    # Agregar el termino de regularización

    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))

    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term

    # Backpropogation

    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)

    # Agregar regularización al gradiente

    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]

    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]

    # ===================== Alterntate solutions =====================
    # my_final_matrix = np.zeros(a3.shape)
    # for c in np.arange(num_labels):
    #    my_final_matrix[:, c] = (np.log(a3[:, c]) * (y == c)) + (np.log(1 - a3[:, c]) * (1 - (y == c)))
    #J = (-1 / m) * np.sum(my_final_matrix)
    # ================================================================

    # ================================================================
    # Unroll gradients
    # grad = np.concatenate([Theta1_grad.ravel(order=order), Theta2_grad.ravel(order=order)])

    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad

In [9]:
lambda_ = 0
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_)
print('Costo en parametros (cargado de ex4weights): %.6f ' % J)

Costo en parametros (cargado de ex4weights): 10.799071 


In [10]:
z = np.array([-1, -0.5, 0, 0.5, 1])
g = sigmoidGradient(z)
print('Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:\n  ')
print(g)

Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:
  
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [11]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    """
    Randomly initialize the weights of a layer in a neural network.

    Parameters
    ----------
    L_in : int
        Number of incomming connections.

    L_out : int
        Number of outgoing connections.

    epsilon_init : float, optional
        Range of values which the weight can take from a uniform
        distribution.

    Returns
    -------
    W : array_like
        The weight initialiatized to random values.  Note that W should
        be set to a matrix of size(L_out, 1 + L_in) as
        the first column of W handles the "bias" terms."""


    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [12]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [13]:
# Define lambda (parámetro de regularización)
lambda_ = 1

# Crea una función de costo abreviada para minimizar
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Ejecuta la optimización
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC')

# Obtiene la solución de la optimización
nn_params = res.x

# Obtiene Theta1 y Theta2 de nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))


<ipython-input-6-e347f585c8bb>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [14]:
def predict(Theta1, Theta2, X):
    """
    Predict the label of an input given a trained neural network
    Outputs the predicted label of X given the trained weights of a neural
    network(Theta1, Theta2)
    """
    # Useful values
    m = X.shape[0]
    num_labels = Theta2.shape[0]

    # You need to return the following variables correctly
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [15]:
pred = predict(Theta1, Theta2, X_test[:100])
accuracy = np.mean(pred == y_test[:100]) * 100
print(pred)
print('Training Set Accuracy: %.2f%%' % accuracy)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 2 2
 2 2 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 1 1 1 1]
Training Set Accuracy: 80.00%


<ipython-input-6-e347f585c8bb>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [16]:
pred = predict(Theta1, Theta2, X[:100])
accuracy = np.mean(pred == y[:100]) * 100
print(pred)
print('Training Set Accuracy: %.2f%%' % accuracy)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 2 2 2 2 1 1 1 2 2 1 2 2 2 1 1 1 1 1
 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 1 1 1 1 1 1 0 2 2 2 2 2 2 2 2 2 1
 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Training Set Accuracy: 76.00%


<ipython-input-6-e347f585c8bb>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
